# Conexión a Bases de datos

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />


- Abrir Anaconda prompt
- Instalar el conector de MySQL para python

    `conda install -c anaconda mysql-connector-python`

In [ ]:
import numpy as np
import pandas as pd
import mysql.connector as sql

In [ ]:
# Database args
db_user = 'intensivo@banistmoserver'
db_passwd = '1234.intenso'
db_host = 'banistmoserver.mysql.database.azure.com'
db_name = 'intensivopython'

In [ ]:
cnx = sql.connect(user=db_user, password=db_passwd, host=db_host, database=db_name)

Usando el cursor de SQL. Ojo que antes de cerrar el cursor tenemos que realizar las operaciones de lectura que necesitemos.

In [ ]:
query= "SHOW TABLES;"

cursor = cnx.cursor(buffered=True)
cursor.execute(query)

for el in cursor:
    print(el)

cursor.close()

Usando el método read_sql de Pandas

In [ ]:
query= "SHOW TABLES;"
tablas=pd.read_sql(query, con=cnx)
tablas

Hagamos un select en la tabla regiones

In [ ]:
query= "SELECT * from regiones;"
regiones=pd.read_sql(query, con=cnx)
regiones

La siguiente consulta está parametrizada. Fíjese cómo se declaran los parámetros dentro de la cadena de la consulta, y cómo después se ingresan, en el comando read_sql de Pandas.

In [ ]:
query= "SELECT * from regiones WHERE region_id BETWEEN %s and %s;"
regiones=pd.read_sql(query, params=(2,8), con=cnx)
regiones

Como verá en la siguiente consulta, estamos utilizando cosas aprendidas en los tutoriales anteriores. Estamos armando dinámicamente la cadena que respresenta una consulta. Específicamente, utilizamos una lista con id's de regiones que nos interesan, lo convertimos a un objeto `array` de numpy (np.array), después aplicamos la función `map` para aplicar la función `str` al array, y finalmente, concatenamos los resultados con el caracter ",". **Todo en una línea**.
¿En cuantas líneas podría hacer esto en Java?

In [ ]:
','.join(["Hola","Mundo","Como","Estan"])

In [ ]:
region_ids=[5,8,10]

query= "SELECT * from regiones WHERE region_id IN ("
query_region= ', '.join(map(str, np.array(region_ids)))
query= query + query_region + ");"
print(query)

regiones=pd.read_sql(query, con=cnx)
regiones

Cerramos la conexión al servidor de MySQL. **¡¡No olvide hacer esto!!**

In [ ]:
cnx.close()